In [2]:
import os

In [1]:
%pwd

'/Users/revanth_reddy/Desktop/AskmeOffers_Coupuon_Acceptance_Analysis_and_Prediction_with_MLOps(Mlflow)/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/revanth_reddy/Desktop/AskmeOffers_Coupuon_Acceptance_Analysis_and_Prediction_with_MLOps(Mlflow)'

In [26]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    iterations: int
    depth: int
    target_column: str

In [27]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [28]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.CatBoostClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            iterations = params.iterations,
            depth = params.depth,
            target_column = schema.name
            
        )

        return model_trainer_config

In [29]:
import pandas as pd
import os
from mlProject import logger
import joblib
from catboost import CatBoostClassifier

In [30]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        cat = CatBoostClassifier(iterations=self.config.iterations, depth=self.config.depth, random_state=42)
        cat.fit(train_x, train_y)

        joblib.dump(cat, os.path.join(self.config.root_dir, self.config.model_name))



In [31]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-05-11 01:59:00,362: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-05-11 01:59:00,364: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-11 01:59:00,366: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-11 01:59:00,366: INFO: common: created directory at: artifacts]
[2024-05-11 01:59:00,367: INFO: common: created directory at: artifacts/model_trainer]
Learning rate set to 0.404581
0:	learn: 0.6275672	total: 62.2ms	remaining: 3.17s
1:	learn: 0.5956880	total: 63.4ms	remaining: 1.58s
2:	learn: 0.5780880	total: 64.4ms	remaining: 1.05s
3:	learn: 0.5659778	total: 65.4ms	remaining: 785ms
4:	learn: 0.5567688	total: 66.4ms	remaining: 624ms
5:	learn: 0.5512447	total: 67.2ms	remaining: 515ms
6:	learn: 0.5449777	total: 67.9ms	remaining: 437ms
7:	learn: 0.5387175	total: 68.7ms	remaining: 378ms
8:	learn: 0.5321193	total: 70.1ms	remaining: 335ms
9:	learn: 0.5282953	total: 71.3ms	remaining: 299ms
10:	learn: 0.5239182	total: 72.4ms	remain